In [14]:
import numpy as np
import pandas as pd
import glob
import os
import scipy.io
%load_ext autoreload
import process_functions as pf
import matplotlib.pyplot as plt
import scipy
import mne
import pickle
from sklearn.model_selection import train_test_split
import sklearn.metrics

d_path = '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory/'
#check

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Split into train + test


In [79]:
meta = pd.read_csv(d_path + 'participants.tsv', sep='\t')
data_l = glob.glob(f'{d_path}/segmented_data/*')
#Define key parameters
fs = 500 #sampling rate Hz
#These may need to be changed as hyper-parameters
low_f = 50 #freq higher will be removed
high_f = 0.1 #freq lower will be removed
chunk_size = (25*fs) #30 seconds
meta
data_l

['/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-2315_sub-072-10.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-1231_sub-038-17.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-197_sub-008-9.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-2407_sub-074-40.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-2752_sub-085-19.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-320_sub-011-23.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-243_sub-009-23.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-1362_sub-042-2.pkl',
 '/Users/dominicburrows/Dropbox/PhD/Analysis/Project/deep_EEGauditory//segmented_data/epoch-2

In [74]:
# Manually split data
#===============================
# 80 train, 10 val, 10 test -> for now
#===============================
#NB no donor can be across multiple groups

#Split into train and test
arr = meta['participant_id'] # Example array
fractions = [0.7, 0.15, 0.15]  # Fractions for splitting train, val, test
seed = 3 #SEED = 3  
train_id, val_id, test_id = pf.split_array(arr, fractions, seed)

#Ensure there's representation from each class 
print('Train = ' + str(np.unique(meta.loc[train_id.index]['Group'],return_counts=True)))
print('Val = ' + str(np.unique(meta.loc[val_id.index]['Group'],return_counts=True)))
print('Test = ' + str(np.unique(meta.loc[test_id.index]['Group'],return_counts=True)))

Train = (array(['A', 'C', 'F'], dtype=object), array([23, 20, 18]))
Val = (array(['A', 'C', 'F'], dtype=object), array([6, 5, 2]))
Test = (array(['A', 'C', 'F'], dtype=object), array([7, 4, 3]))


In [81]:
[i for i in data_l if train_id.values in i]

TypeError: 'in <string>' requires string as left operand, not numpy.ndarray

In [82]:
train_id.values

array(['sub-068', 'sub-074', 'sub-010', 'sub-070', 'sub-036', 'sub-082',
       'sub-087', 'sub-050', 'sub-071', 'sub-007', 'sub-053', 'sub-026',
       'sub-086', 'sub-013', 'sub-009', 'sub-035', 'sub-005', 'sub-069',
       'sub-018', 'sub-037', 'sub-046', 'sub-063', 'sub-014', 'sub-059',
       'sub-024', 'sub-012', 'sub-077', 'sub-058', 'sub-065', 'sub-055',
       'sub-033', 'sub-032', 'sub-017', 'sub-028', 'sub-079', 'sub-048',
       'sub-066', 'sub-054', 'sub-062', 'sub-056', 'sub-051', 'sub-019',
       'sub-080', 'sub-081', 'sub-084', 'sub-029', 'sub-085', 'sub-078',
       'sub-031', 'sub-076', 'sub-038', 'sub-049', 'sub-016', 'sub-060',
       'sub-044', 'sub-008', 'sub-088', 'sub-043', 'sub-030', 'sub-034',
       'sub-047'], dtype=object)

In [12]:
! ls {d_path}/segmented_data | wc -l

    2837


# Loader functions

#

In [ ]:
#===================================================
#Filter out unwanted frequencies, segment and save
#===================================================



superid_count, subid_count = 1,1 #define counters for full dataset id and individual id
for m in range(len(meta)):
    subid_count=1
    id = meta['participant_id'][m]
    sex = meta['Gender'][m]
    age = meta['Age'][m]
    group = meta['Group'][m]
    mmse = meta['MMSE'][m]
    path = d_path + 'derivatives/' + id + '/eeg/' + id + '_task-eyesclosed_eeg.set'
    
    #Load in
    dic = pf.set_load(path, id, sex, age, group, mmse)
    raw = dic['data']

    #Apply lowpass filter > 50Hz
    filt = pf.lowpassfilt(raw, low_f, fs)
    #Apply highpass filter < 0.1Hz
    filt = pf.highpassfilt(filt, high_f, fs)
    
    dic['data'] = filt

    #Segment
    bins = pf.split_array_into_bins(dic['data'], chunk_size)

    for b in bins:
        sub_dic={}
        sub_dic['id'] = dic['id']
        sub_dic['sex'] = dic['sex']
        sub_dic['age'] = dic['age']
        sub_dic['group'] = dic['group']
        sub_dic['ch_names'] = dic['ch_names']
        sub_dic['data'] = b
        name = f"""epoch-{superid_count}_{sub_dic['id']}-{subid_count}.pkl""" #give unique identifier and sampleid
        with open(d_path+'/segmented_data/'+name, 'wb') as pickle_file:
            pickle.dump(sub_dic, pickle_file)
        subid_count+=1
        superid_count+=1
    